# Setup

In [25]:
%load_ext autoreload
%autoreload 2

# gee functions
import ee
import folium
import geemap
import gee_utils
# Authenticate to Earth Engine
ee.Authenticate()
# Initialize Earth Engine instance
ee.Initialize(project="reeftruth")
#!!! (for readme)
# https://stackoverflow.com/questions/78374548/no-authentication-box-appears-when-authenticating-google-earth-engine-gee-pyth

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = gee_utils.add_ee_layer

# Vis params and AOIs

In [12]:
### Visualisation params
# harmonized sentinel-2 bands
sVisParams = {
  "min": 0,
  "max": 3000,
  "gamma": 1.5,
  "bands": ["B4", "B3", "B2"]
}
# Define the bands to be used
sBands = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A"]
# PRSIM benthic classes
prismVisParams = {
  "min": 0,
  "max": 1,
  "color": ["red", "green", "blue"],
};

### AOI
# Define bounds
swains_bounds = [[151.9006776532874, -22.425424797331246],
          [152.83451554391243, -22.425424797331246],
          [152.83451554391243, -21.408761467649015],
          [151.9006776532874, -21.408761467649015],
          [151.9006776532874, -22.425424797331246]]
swains_aoi = ee.Geometry.Polygon(swains_bounds)


In [55]:
import ee
import geemap

# set AOI
aoi_geom = swains_aoi

# Define the Sentinel-2 Image Collection and Cloud Collection
sentinel_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
sentinel_clouds = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Filter satellite timeseries for best image
sentinel_colour = sentinel_collection.filterDate("2022-01-01", "2023-01-01") \
    .filterBounds(aoiGeom)  \
    .linkCollection(sentinel_clouds, ["cs_cdf"]) \
    .map(lambda image: image.updateMask(image.select("cs_cdf").gt(0.6)))

composite = sentinel_colour.median()

# Calculate NDWI to mask land
ndwi = composite.expression(
    '(GREEN - NIR) / (GREEN + NIR)',
    {
        'GREEN': composite.select('B3'),
        'NIR': composite.select('B8')
    }
)

# Sunglint correction and water mask
water_image = composite.updateMask(ndwi.gt(0.1)).select(['B4', 'B3', 'B2']).subtract(composite.select('B8'))
sunglint_correction = composite.select(['B4', 'B3', 'B2']).subtract(composite.select('B8'))

# Add layers to the map
Map = geemap.Map(height='1000px')
Map.addLayer(water_image.clip(aoiGeom), sVisParams, 'satellite-derived land mask (NDWI)')
Map.addLayer(composite.clip(aoiGeom), sVisParams, 'sentinel composite')
Map.addLayer(sunglint_correction.clip(aoiGeom), sVisParams, 'top of water reflectance', False)

# Uncomment below to use the RF-based mask
# masked_img = gee_utils.landmask(composite.clip(aoiGeom))
# Map.addLayer(masked_img, sVisParams, 'satellite-derived land mask (RF)', False)
# Map

In [60]:
# get bands from swains
swains_flight_bounds = swains.get("system:footprint").getInfo().get("coordinates")
out = ee.Geometry.Polygon(swains_flight_bounds)

In [66]:
# create sunglint_correction cropped to swains bounds
swains_sunglint_correction = sunglint_correction.clip(out)
Map.addLayer(swains_sunglint_correction, sVisParams, 'top of water reflectance (swains bounds)', False)

# PRISM data

In [67]:
swains = ee.Image("projects/reeftruth/assets/PRISM/Swains").rename(['coral','algae','sand'])

Map.addLayer(swains, prismVisParams, 'PRISM benthic classes')
Map.centerObject(aoiGeom, 11)
Map

Map(bottom=589846.0, center=[-21.91714700361253, 152.36759659859993], controls=(WidgetControl(options=['positi…

In [ ]:
# TODO:
# - add depth retrieval
# - mask depth (else may see classes which haven't been classified)
# - add depth profile to map
# - split into test and train sets
# - train ee RF model
# - export relevant imagery to drive
# - train sklearn RF model locally (https://geemap.org/notebooks/46_local_rf_training/)
# - transfer model to EE